In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression

In [9]:
# Data has no none entry so it is not necessary to drop any rows
# Transform categorical string columns to numerical values and keep a dictionary to map them back
def transform_strings_to_numerical(data):
    """
    This function transforms all string values in the dataframe to numerical values using the LabelEncoder from sklearn.
    Args:
        data : Our dataframe which we want to modify

    Returns:
        data : Our modified dataframe
        transform_data : A dictionary containing the mapping from the original string values to the numerical values
    """

    transform_data = {}
    for column in data.columns:
        # If data type is an object, for example a string, we want to convert the column to numerical values
        if data[column].dtype == 'object' and data[column].dtype != 'datetime64[ns]':
            le = LabelEncoder()
            data[column] = le.fit_transform(data[column])
            # Save the mapping in a dictionary
            transform_data[column] = dict(zip(le.classes_, le.transform(le.classes_)))
    return data, transform_data

In [3]:
def transform_numerical_to_string(data,transform_data):
    """
    This function transforms all numerical values in the dataframe back to the original string values using the LabelEncoder from sklearn.
    Args:
        data : Our dataframe which we want to modify
        transform_data : A dictionary containing the mapping from the original string values to the numerical values

    Returns:
        data : Our modified dataframe
    """
    for column, mapping in transform_data.items():
        #print(f"Mapping for column {column}: {mapping}")
        #if one of the mapped columns is not in the data anymore, we skip it
        if column not in data.columns:
            continue
        # need to reverse the mapping to map back to the original string values
        # this simply swaps the keys and values in the dictionary
        reverse_mapping = {v: k for k, v in mapping.items()}
        # now we can convert back to the original string values
        data[column] = data[column].map(reverse_mapping)
    return data

In [14]:
def get_data():
    """
    Returns the data from the csv file and transforms the categorical values to numerical values
    """
    # read in the data from the csv file
    data = pd.read_csv('data/kickstarter_projects.csv')
    # transform the categorical values to numerical values

    data = data[(data["State"] == "Successful") | (data["State"] == "Failed")]
    data["Deadline"] = pd.to_datetime(data["Deadline"],format='%Y-%m-%d')
    data["Launched"] = pd.to_datetime(data["Launched"],format='%Y-%m-%d %H:%M:%S')
    data["Duration"] = (data["Deadline"] - data["Launched"]).dt.days


    data, transform_data = transform_strings_to_numerical(data)



    """
    We have converted Deadline and Launched to DateTime objects and calculated the duration in days
    We also, at least for now, drop all live or suspended or canceled projects
    
    """
    #return the data and the transformation_data in case we want to transform the data back
    return data, transform_data

In [83]:
data, transform_data= get_data()
data.drop("Name", axis=1, inplace=True)
data.head(20)

data = transform_numerical_to_string(data,transform_data)
data["State"].groupby(data["State"]).count()

Series([], Name: State, dtype: int64)

In [13]:
def get_original_data():
    """
    Returns the original data without any modifications
    """
    # read in the data from the csv file
    data = pd.read_csv('data/kickstarter_projects.csv')


    data = data[(data["State"] == "Successful") | (data["State"] == "Failed")]
    data["Deadline"] = pd.to_datetime(data["Deadline"],format='%Y-%m-%d')
    data["Launched"] = pd.to_datetime(data["Launched"],format='%Y-%m-%d %H:%M:%S')
    data["Duration"] = (data["Deadline"] - data["Launched"]).dt.days



    """
    We have converted Deadline and Launched to DateTime objects and calculated the duration in days
    We also, at least for now, drop all live or suspended or canceled projects
    
    """
    #return the data
    return data

In [85]:
get_original_data().head(20)

,ID,Name,Category,Subcategory,Country,Launched,Deadline,Goal,Pledged,Backers,State,Duration


In [86]:
def remove_outlier(data,columns,threshold=3):
    """
    This function removes outliers from the data based on the threshold
    Args:
        data : Our dataframe which we want to modify
        columns : The columns which we want to check for outliers
        threshold : The threshold which we use to determine if a value is an outlier
        Multiplied by the standard deviation of the column to determine the range of values which are not outliers
        I advise setting the threshold to 3

    Returns:
        data : Our modified dataframe
    """
    
    for column in columns:
        data = data[np.abs(data[column]-data[column].mean()) <= (threshold*data[column].std())]
    return data

In [21]:
def logistic_regression(X_train,X_test,y_train,y_test,metric="accuracy",verbose=False,norm="l2",max_iter=1000,C=1.0):
    """
    This function performs logistic regression on the data and returns the accuracy of the model
    Necessary Arguments:
        X_train : The training data
        X_test : The test data
        y : The target values
        y_train : The target values for the training data

    Optional Arguments:
        metric : The metric to calculate the model performance, Options: "accuracy", "precision", "recall", "f1"
        verbose : If True, the function will print the metric of the model
        norm : The norm to use for the logistic regression
        max_iter : The maximum number of iterations for the logistic regression
        C : The regularization parameter for the logistic regression

    Returns:
        metric : The number of the metric specified in the arguments
    """

    # create a logistic regression model
    model = LogisticRegression(max_iter=max_iter,penalty=norm,C=C)

    # fit the model to the training data
    model.fit(X_train, y_train)

    # predict the target values for the test data
    y_pred = model.predict(X_test)

    if verbose==True:
        print(f"Model Coefficients: {model.coef_}")
        print(f"Model Intercept: {model.intercept_}")
        print(f"Model Score: {y_pred}")

    metric_value = 0
    # calculate the metric of the model
    if metric == "accuracy":
        metric_value = accuracy_score(y_test, y_pred)
    if metric == "precision":
        metric_value = precision_score(y_test, y_pred)
    if metric == "recall":
        metric_value = recall_score(y_test, y_pred)
    if metric == "f1":
        metric_value = f1_score(y_test, y_pred)
    return metric_value

In [18]:

def grid_search(model,parameters,metric,X_train,X_test,y_train,y_test):
    from itertools import product
    """
    
    Perform grid search for the given machine learning model and hyperparameters to find the best hyperparameters

    Parameters:
    model - The machine learning model function to use
    param_grid - The hyperparameters to test, given in form of a list of dictionaries
    metric - A string specifying the metric to use for evaluation
    X_train, X_test, y_train, y_test - The training and test data

    Returns: A pandas Dataframe containing the hyperparameters and the corresponding metric value, 
    sorted by the metric value in descending order

    """
    # Create all possible permutations of the hyperparameters, so if a={1,2} and b={3,4} we get [{1,3},{1,4},{2,3},{2,4}]
    keys, values = zip(*parameters.items())
    permutations = [dict(zip(keys, v)) for v in product(*values)]

    # Create a list to store the results
    results = []

    for params in permutations:
        # feed the model with the hyperparameters
        # ** unpacks the dictionary into the form dict[key]=value -> key = value
        metric_value = model(X_train,X_test,y_train,y_test,**params)

        # Append the results to the list
        results.append((params, metric_value))

    # After the loop is done, we sort the results by the metric value
    results.sort(key=lambda x: x[1], reverse=True)

    results = pd.DataFrame(results, columns=['Parameters', metric])

    return results




    

In [23]:
from sklearn.model_selection import train_test_split
# test out the functions
data,transform_data = get_data()
print(data.head())

y = data['State']
X = data.drop(['State','Launched','Deadline'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 1337)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

           ID    Name  Category  Subcategory  Country            Launched   
0  1860890148  115746         5           52       21 2009-04-21 21:02:48  \
1   709707365   56298         6          129       21 2009-04-23 00:07:53   
2  1703704063  323842         0           70       21 2009-04-24 21:52:03   
3      727286  192565        13          131       21 2009-04-25 17:36:21   
4  1622952265  200031         5           52       21 2009-04-27 14:10:39   

    Deadline   Goal  Pledged  Backers  State  Duration  
0 2009-05-31   1000      625       30      0        39  
1 2009-07-20  80000       22        3      0        87  
2 2009-05-03     20       35        3      1         8  
3 2009-07-14     99      145       25      1        79  
4 2009-05-26   1900      387       10      0        28  


((265169, 9), (66293, 9), (265169,), (66293,))

In [24]:
#import warnings
#
# warnings.filterwarnings("ignore", category=ConvergenceWarning)

hyperparameters = {"C":[0.1,1.0,10.0],"max_iter":[10,100,1000]}
results = grid_search(logistic_regression,hyperparameters,"accuracy",X_train,X_test,y_train,y_test)

results.head()


c:\Users\tobia\anaconda3\envs\env\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\tobia\anaconda3\envs\env\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

,Parameters,accuracy
0,"{'C': 10.0, 'max_iter': 1000}",0.999110
1,"{'C': 0.1, 'max_iter': 1000}",0.999095
2,"{'C': 1.0, 'max_iter': 1000}",0.999095
3,"{'C': 10.0, 'max_iter': 100}",0.998989
4,"{'C': 0.1, 'max_iter': 100}",0.998884
